In [12]:
# Dependencies and Setup
import pandas as pd
import requests
# import numpy as np
import io
import ipyleaflet
import json
# from pathlib import Path
# from pprint import pprint
from ipyleaflet import Map, Marker, LayersControl, SearchControl, FullScreenControl, WidgetControl, GeoJSON, LayerGroup, Popup
# from geopy.exc import GeocoderTimedOut 
# from geopy.geocoders import Nominatim 
# from ipywidgets import link, FloatSlider
from branca.colormap import linear
from pandas_geojson import to_geojson
from ipywidgets import HTML, widgets
from shapely.geometry import shape

In [2]:
# Downloading the csv file from the GitHub repository
url = "https://raw.githubusercontent.com/brown016340/Project---3/benjamin/data/coords.csv"
download = requests.get(url).content

# Reading the downloaded content and turning it into a pandas dataframe
coords_df = pd.read_csv(io.StringIO(download.decode('utf-8')))

# Printing out the first 5 rows of the dataframe
coords_df.head()

,Country,Latitude,Longitude
0,Switzerland,46.798562,8.231974
1,Iceland,64.984182,-18.105901
2,Denmark,55.670249,10.333328
3,Norway,61.152939,8.787665
4,Canada,61.066692,-107.991707


In [3]:
# Downloading the csv file from the GitHub repository
url = "https://raw.githubusercontent.com/brown016340/Project---3/benjamin/data/2015.csv"
download = requests.get(url).content

# Reading the downloaded content and turning it into a pandas dataframe
happy15_df = pd.read_csv(io.StringIO(download.decode('utf-8')))

# Printing out the first 5 rows of the dataframe
happy15_df.head()

,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Switzerland,Western Europe,1,7.587,0.03411,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738
1,Iceland,Western Europe,2,7.561,0.04884,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201
2,Denmark,Western Europe,3,7.527,0.03328,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204
3,Norway,Western Europe,4,7.522,0.03880,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531
4,Canada,North America,5,7.427,0.03553,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176


In [4]:
# Join the dataframes
data_df = pd.merge(happy15_df, coords_df, on='Country', how='inner')
data_df.head()

,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual,Latitude,Longitude
0,Switzerland,Western Europe,1,7.587,0.03411,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738,46.798562,8.231974
1,Iceland,Western Europe,2,7.561,0.04884,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201,64.984182,-18.105901
2,Denmark,Western Europe,3,7.527,0.03328,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204,55.670249,10.333328
3,Norway,Western Europe,4,7.522,0.03880,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531,61.152939,8.787665
4,Canada,North America,5,7.427,0.03553,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176,61.066692,-107.991707


In [5]:
def load_data(url, filename, file_type):
    r = requests.get(url)
    with open(filename, 'w') as f:
        f.write(r.content.decode("utf-8"))
    with open(filename, 'r') as f:
        return file_type(f)

In [6]:
data = load_data(
    'https://raw.githubusercontent.com/brown016340/Project---3/benjamin/data/2015.geo.json',
    '2015.geo.json',
     json.load)

In [15]:
# Load the choropleth map
layer = ipyleaflet.Choropleth(
    geo_data= data,
    choro_data=dict(zip(data_df['Country'].str[:2].str.upper().tolist(), data_df['Happiness Score'].tolist())),
    colormap=linear.YlOrRd_04,
    border_color='black',
    style={'fillOpacity': 0.8, 'dashArray': '5, 5'},
    name= '2015'
)
m = ipyleaflet.Map(center = (0,0), zoom = 1)
m.add(layer)

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [16]:
control = LayersControl(position='topright')
m.add(control)

Map(bottom=456.0, center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

In [ ]:
m.add(FullScreenControl())

In [ ]:
# m.save('C:/Users/Brown/Documents/my_map.html', title='My Map')

In [14]:
m.clear()

Map(bottom=384.0, center=[45.089035564831036, 1.40625], crs={'name': 'EPSG3857', 'custom': False}, default_sty…

In [17]:
# Search in a GeoJSON 

countries = GeoJSON(data=data)

# m = Map(center=[61.9389, 69.7851], zoom=2)

search = SearchControl(
    position="topleft",
    layer=LayerGroup(layers=(countries,)),
    property_name="Country",
)
m.add(search)
m

Map(bottom=456.0, center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

In [18]:
def add_popup(feature, **kwargs):
    for layer in m.layers:
        if isinstance(layer, Popup):
            m.remove_layer(layer)
    p = shape(feature["geometry"])
    center = p.centroid.y, p.centroid.x
    properties = feature["properties"]
    message = HTML()
    message.value = f"""<h4>{properties["Country"]}</h4>
                        <b>Happiness Score: {properties["Happiness Score"]}
                     """
    popup = Popup(
        location=center,
        child=message,
        close_button=True,
        auto_close=False,
        close_on_escape_key=False,
    )

    m.add(popup)


search.on_feature_found(add_popup)